In [2]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from BI import bi

In [ ]:

import numpyro
import numpy as np
import pandas as pd
import jax.numpy as jnp
import jax
import tensorflow_probability.substrates.jax as tfp
tfd = tfp.distributions
import tensorflow as tf
import jax.random as random

m = bi(platform='cpu')
data_path = os.path.dirname(os.path.abspath("")) + "/BI/resources/data/"


/home/sosa/work/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/sosa/work/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/sosa/work/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compa

jax.local_device_count 16


# Code block 1

In [ ]:

numpyro.distributions.Normal(0, 1).sample(random.PRNGKey(0), sample_shape=(5,))

tfd.Normal(loc=1.0, scale=1.0).sample(sample_shape=(5,), seed=key)

m.dist.normal(0, 1, name = "mu", shape = (5,), sample=True)

Array([-0.20584214, -0.78476578,  1.81608667,  0.18784401,  0.08086788],      dtype=float64)

# Code block 2

In [ ]:
import numpyro.distributions as dist
import tensorflow_probability as tf
import tensorflow as tf
import jax.numpy as jnp
import jax.random as random
import numpyro

from BI import bi
tfd = tfp.distributions
m = bi()

# Numpyro version of random centered effect ----------------------------------------
a = numpyro.distributions.Normal(5, 2).sample(random.PRNGKey(0), sample_shape=(1,))
b = numpyro.distributions.Normal(-1, 0.5).sample(random.PRNGKey(0), sample_shape=(1,) )
sigma_cafe = numpyro.distributions.Exponential(1).sample(random.PRNGKey(0), sample_shape=(2,))
sigma = numpyro.distributions.Exponential(1).sample(random.PRNGKey(0))
Rho = numpyro.distributions.LKJ(2, 2).sample(random.PRNGKey(0))
cov = jnp.outer(sigma_cafe, sigma_cafe) * Rho
a_cafe_b_cafe =numpyro.distributions.MultivariateNormal(jnp.stack([a, b]), cov).sample(random.PRNGKey(0), sample_shape=(5,))
a_cafe, b_cafe = a_cafe_b_cafe[:, 0], a_cafe_b_cafe[:, 1]

# TFP version of random centered effect ----------------------------------------
a = tfd.Normal(5., 2.).sample()
b = tfd.Normal(-1., 0.5).sample()
sigma_cafe = tfd.Exponential(rate=1.).sample(sample_shape=(2,))
sigma = tfd.Exponential(rate=1.).sample()
Rho = tfd.LKJ(dimension=2, concentration=2.).sample()
cov = tf.einsum('i,j->ij', sigma_cafe, sigma_cafe) * Rho
a_cafe_b_cafe = tfd.MultivariateNormalFullCovariance(
    loc=tf.stack([a, b], axis=-1),   # shape (2,)
    covariance_matrix=cov
).sample(sample_shape=(5,))          # shape (5, 2)
a_cafe, b_cafe = a_cafe_b_cafe[:, 0], a_cafe_b_cafe[:, 1]

# BI version of random centered effect----------------------------------------
a = m.dist.normal(5, 2,  name = 'a',sample=True)
b = m.dist.normal(-1, 0.5, name = 'b',sample=True)
sigma = m.dist.exponential( 1,  name = 'sigma',sample=True)
varying_intercept, varying_slope = m.effects.varying_intercept_slope(
    N_group = 20,
    group = jnp.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]), # dummy group
    global_intercept= a,
    global_slope= b,
    group_name = 'cafe',
    sample=True
)

jax.local_device_count 16


# Code block 3

In [ ]:
def model(N_id, idx,  result_outcomes,
          focal_individual_predictors, 
          target_individual_predictors):

    # Intercept 
    intercept = bi.dist.normal(
        logit(0.1/jnp.sqrt(N_id)), 
        2.5, shape=(1,), name = 'intercept'
    )

    # Sender receiver  ----------------------
    N_var = focal_individual_predictors.shape[0]
    N_id = focal_individual_predictors.shape[1]   
    focal_effects = m.dist.normal(0, 1,  name = 'focal_effects')
    target_effects = m.dist.normal( 0, 1,  name = 'target_effects')
    terms = jnp.stack([
        focal_effects @ focal_individual_predictors,
        target_effects @  target_individual_predictors
        ], axis = -1)
    sr_raw =  m.dist.normal(0, 1, shape=(2, N_id), name = 'sr_raw')
    sr_sigma =  m.dist.exponential( 1, shape= (2,), name = 'sr_sigma')
    sr_L = m.dist.lkjcholesky(2, 2, name = "sr_L")
    rf = (((sr_L @ sr_raw).T * sr_sigma))
    ids = jnp.arange(0,sr_effects.shape[0])
    edgl_idx = m.net.vec_node_to_edgle(jnp.stack([ids, ids], axis = -1))
    sender = sr_effects[edgl_idx[:,0],0] + sr_effects[edgl_idx[:,1],1]
    receiver = sr_effects[edgl_idx[:,1],0] + sr_effects[edgl_idx[:,0],1]
    sr =  jnp.stack([sender, receiver], axis = 1)

    # dyadic effects ------------------------------------------
    m.net.mat_to_edgl(dyadic_effect_mat)
    dr_raw =  m.dist.normal(0, 1, shape=(2,N_dyads), name = 'dr_raw')
    dr_sigma = m.dist.exponential(1,  name = 'dr_sigma' )
    dr_L = m.dist.lkjcholesky(2, 2, name = 'dr_L')
    dr_rf = deterministic('dr_rf', (
        ((dr_L @ dr_raw).T * jnp.repeat(dr_sigma, 2))
        ))

    dyad_effects = dist.normal(0, 1, 
        name= 'dyad_effects', shape = (dyadic_predictors.ndim - 1,
    ))
    dr = dyad_effects * dyadic_predictors

    # Likelihood                                                       
    m.dist.poisson(jnp.exp(intercept + sr + dr), obs=result_outcomes)  

# Code block 4

In [ ]:
def model(N_id, idx,  result_outcomes,
          focal_individual_predictors, 
          target_individual_predictors):

    # Intercept 
    intercept = m.dist.normal(
        logit(0.1/jnp.sqrt(N_id)), 
        2.5, shape=(1,), name = 'intercept'
    )

    # SR 
    sr =  m.net.sender_receiver(
        focal_individual_predictors,
        target_individual_predictors
    )

    # Dyadic  
    dr = m.net.dyadic_effect(shape = idx.shape[0])

    # Likelihood                                                       
    m.dist.poisson(jnp.exp(intercept + sr + dr), obs=result_outcomes)  

# Code block 5

In [ ]:
model <- function(N_id, idxShape, result_outcomes,
                focal_individual_predictors, target_individual_predictors){

  x=0.1/jnp$sqrt(N_id)
  tmp=jnp$log(x / (1 - x))
  
  # Intercept 
  intercept = bi.dist.normal(tmp, 2.5, shape=c(1), name = 'block')
  
  # SR 
  sr =  m$net$sender_receiver(
            focal_individual_predictors,
            target_individual_predictors
        )
  
  # Dyadic  
  dr = m$net$dyadic_effect(shape = c(idxShape))

  # Likelihood                                                       
  m$poisson(jnp$exp(intercept + sr + dr), obs=result_outcomes)  